In [1]:
#pip install torch

In [2]:
#pip install transformers

In [3]:
#pip install SentencePiece

In [15]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
import torch

tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
model = XLNetLMHeadModel.from_pretrained('xlnet-large-cased')

In [16]:
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing."""

In [17]:
import random
import numpy as np

def choose_from_top(probs, k=5, sample_size=1):
    ind = np.argpartition(probs, -k)[-k:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob)
    choice = np.random.choice(k, sample_size, p = top_prob, replace=False)
    token_ids = ind[choice]
    return token_ids

In [18]:
sent = "If sentence is not as long as the desired sentence length,"
topk = 10
n = 20
temperature = 0.7

In [19]:
model.eval()
if torch.cuda.is_available(): model.to('cuda')

sent_tokens = tokenizer.encode(sent, add_special_tokens=False)
print(sent_tokens)
mask_tokens = tokenizer.encode('<mask>', add_special_tokens=False)
print(mask_tokens)
padding_tokens = tokenizer.encode(PADDING_TEXT, add_special_tokens=False)
print(padding_tokens)

[183, 3833, 27, 50, 34, 206, 34, 18, 7181, 3833, 2486, 19]
[6]
[67, 2840, 19, 18, 1484, 20, 965, 29077, 8719, 1273, 21, 45, 273, 17, 10, 15048, 28, 27511, 21, 4185, 11, 41, 2444, 9, 32, 1025, 20, 8719, 26, 23, 673, 966, 19, 29077, 20643, 27511, 20822, 20643, 19, 17, 6616, 17511, 18, 8978, 20, 18, 777, 9, 19233, 1527, 17669, 19, 24, 673, 17, 28756, 150, 12943, 4354, 153, 27, 442, 37, 45, 668, 21, 24, 256, 20, 416, 22, 2771, 4901, 9, 12943, 4354, 153, 51, 24, 3004, 21, 28142, 23, 65, 20, 18, 416, 34, 24, 2958, 22947, 9, 1177, 45, 668, 3097, 13768, 23, 103, 28, 441, 148, 48, 20522, 19, 12943, 4354, 153, 12860, 34, 18, 326, 27, 17492, 684, 21, 6709, 9, 8585, 123, 266, 19, 12943, 4354, 153, 6872, 24, 3004, 20, 18, 9225, 2198, 19, 12717, 103, 22, 401, 24, 6348, 9, 12943, 4354, 153, 1068, 2768, 2286, 19, 33, 104, 19, 176, 24, 9313, 19, 20086, 28, 45, 10292, 9]


In [21]:
for i in range(n):
    input = sent_tokens + mask_tokens
    target_id1 = -len(input)
    target_id2 = -1

    input_ids = torch.tensor(padding_tokens).unsqueeze(0)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[0, :, [target_id1, target_id2]] = 1.0

    target_mapping = torch.zeros((1, 2, input_ids.shape[1]), dtype=torch.float) 
    target_mapping[0, 0, target_id1] = 1.0
    target_mapping[0, 1, target_id2] = 1.0
    #print(target_mapping)

    if torch.cuda.is_available():
        input_ids_tensor = input_ids.to("cuda")
        target_mapping_tensor = target_mapping.to("cuda")
        perm_mask_tensor = perm_mask.to("cuda")
    else:
        input_ids_tensor = input_ids
        target_mapping_tensor = target_mapping
        perm_mask_tensor = perm_mask

    with torch.no_grad():
        outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)

    predicted_tokens = []
    
    for j in range(2):
        probs = torch.nn.functional.softmax(outputs[0][0][j]/temperature, dim = 0).to('cpu').numpy()
        predicted_tokens.append(choose_from_top(probs, k=topk, sample_size=4))
    tok = predicted_tokens[1][0]
    sent_tokens = sent_tokens + [tok]
    
print(tokenizer.decode(sent_tokens))

If sentence is not as long as the desired sentence length, or in addition to the sentence length, then the sentence is not considered a sentence. A sentence is.. and....s.....s when. on...
